In [1]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import torch
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
max_words = 10000
embedding_dim = 50
num_epochs = 5

In [3]:
# (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# x_train = pad_sequences(train_data, maxlen=100)
# x_test = pad_sequences(test_data, maxlen=100)

# y_train = torch.Tensor(train_labels)
# y_test = torch.Tensor(test_labels)

# x_train_np = np.array(x_train)
# y_train_np = np.array(y_train)
# x_test_np = np.array(x_test)
# y_test_np = np.array(y_test)

In [4]:
def compile_RNN(recurrent_units):
    model_rnn = Sequential()
    model_rnn.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=100))
    model_rnn.add(SimpleRNN(units=recurrent_units))
    model_rnn.add(Dense(units=1, activation='sigmoid'))

    model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model_rnn.fit(x_train_np, y_train_np, epochs=num_epochs, batch_size=32, validation_data=(x_test_np, y_test_np), verbose=0)

    return model_rnn

In [5]:
def compile_LSTM(recurrent_units):
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=100))
    model_lstm.add(LSTM(units=recurrent_units))
    model_lstm.add(Dense(units=1, activation='sigmoid'))

    model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model_lstm.fit(x_train_np, y_train_np, epochs=num_epochs, batch_size=32, validation_data=(x_test_np, y_test_np), verbose=0)
    return model_lstm

In [6]:
def get_metrics(model, model_name, recurrent_units, max_sequence_length):
    y_pred = model.predict(x_test_np, verbose=0)
    y_pred = (y_pred > 0.5).astype(int)

    print("--------------------------------------------------------")
    print(f"Model: {model_name}; Recurrent Units: {recurrent_units}; max_sequence_length: {max_sequence_length}")
    print(f"Accuracy: {accuracy_score(y_test_np, y_pred)}")
    print(f"Precision: {precision_score(y_test_np, y_pred)}")
    print(f"Recall: {recall_score(y_test_np, y_pred)}")
    print(f"F1-score: {f1_score(y_test_np, y_pred)}")

In [7]:
recurrent_units_to_test = [16, 64, 128]
max_sequence_lengths = [1, 100]

In [8]:
for recurrent_units in recurrent_units_to_test:
    for max_sequence_length in max_sequence_lengths:
    
        (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

        x_train = pad_sequences(train_data, maxlen=max_sequence_length)
        x_test = pad_sequences(test_data, maxlen=max_sequence_length)

        y_train = torch.tensor(train_labels)
        y_test = torch.tensor(test_labels)

        x_train_np = np.array(x_train)
        y_train_np = np.array(y_train)
        x_test_np = np.array(x_test)
        y_test_np = np.array(y_test)


        model_rnn = compile_RNN(recurrent_units)
        model_lstm = compile_LSTM(recurrent_units)
        
        get_metrics(model_rnn, "rnn model", recurrent_units, max_sequence_length)
        get_metrics(model_lstm, "lstm model", recurrent_units, max_sequence_length)

--------------------------------------------------------
Model: rnn model; Recurrent Units: 16; max_sequence_length: 1
Accuracy: 0.59744
Precision: 0.5829361296472831
Recall: 0.68488
F1-score: 0.6298094607518576
--------------------------------------------------------
Model: lstm model; Recurrent Units: 16; max_sequence_length: 1
Accuracy: 0.59672
Precision: 0.5974057363841444
Recall: 0.5932
F1-score: 0.5952954399486191
--------------------------------------------------------
Model: rnn model; Recurrent Units: 16; max_sequence_length: 100
Accuracy: 0.81976
Precision: 0.8054409292373529
Recall: 0.8432
F1-score: 0.8238880637848824
--------------------------------------------------------
Model: lstm model; Recurrent Units: 16; max_sequence_length: 100
Accuracy: 0.83612
Precision: 0.8164494991338405
Recall: 0.8672
F1-score: 0.8410598595647282
--------------------------------------------------------
Model: rnn model; Recurrent Units: 64; max_sequence_length: 1
Accuracy: 0.59364
Precision: 0

Analiza danych prezentuje wyniki metryk dla modeli rekurencyjnych RNN i LSTM o różnych rozmiarach jednostek rekurencyjnych oraz maksymalnych długości sekwencji.

Porównanie modeli RNN i LSTM:
Dla krótkich sekwencji, modele LSTM mają tendencję do osiągania lepszych wyników niż modele RNN pod względem wszystkich metryk.
Dla długich sekwencji, różnice między modelami RNN a LSTM są bardziej zróżnicowane, ale ogólnie modele LSTM zdają się osiągać lepsze wyniki.

Wpływ liczby jednostek rekurencyjnych:
Dla obu typów modeli (RNN i LSTM), zwiększenie liczby jednostek rekurencyjnych z 16 do 64 wydaje się poprawiać wyniki dla wszystkich metryk, zwłaszcza dla długich sekwencji.
Dla dalszego zwiększenia liczby jednostek rekurencyjnych z 64 do 128, wyniki są zróżnicowane, ale niekoniecznie poprawiają się we wszystkich przypadkach.

Wpływ długości sekwencji:
Dla obu typów modeli i różnych rozmiarów jednostek rekurencyjnych, zwiększenie maksymalnej długości sekwencji z 1 do 100 powoduje znaczną poprawę wyników we wszystkich metrykach.
W przypadku długich sekwencji, modele LSTM często osiągają lepsze wyniki niż modele RNN.

Najlepsze wyniki:
Najlepsze wyniki zdają się być osiągane przez modele LSTM z 128 jednostkami rekurencyjnymi i długimi sekwencjami.
Warto zauważyć, że modele te osiągają wysoką dokładność, precyzję, czułość i F1-score, co sugeruje, że są one skuteczne w rozwiązaniu danego problemu.

Wnioski:
Modele LSTM jako specjalistyczne modele RNN lepiej radzą sobię z tym zadaniem. 
Zwiększenie liczby jednostek rekurencyjnych oraz długości sekwencji może poprawić wyniki, ale należy zwrócić uwagę na ewentualne wzrosty złożoności obliczeniowej.